#### LendingClub -classification(purpose of new client loan status predict 1 은 미래예측이 가능하나 bad loan 진단력이 약하고 predict 2는 상환률을 변수로 사용하여 분류정확도가 완벽하나 미래의 상환정도를 예측할수가 없다. 여기선 na가 많아 삭제된 변수중 bad loan에 관련된 변수를 더 활용할 방법을 찾던지 아니면 bad loan 데이터가 너무 작으므로 늘리는 방법을 찾는다.

#### current 상태를 제외하고 good,bad 확정인 과거데이터를 학습하여 분류기준을 찾는 모델이다. 그렇기 때문에 'repayment_rate'( total_pymnt / funded_amnt) 성격이 결과변수인 성격이 있어서 제외하였다. 정확도는 상대적으로 떨어져도 의미있는 분석모델.  

In [ ]:
# Import our libraries we are going to use for our data analysis.
#import tensorflow as tf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Plotly visualizations
from plotly import tools
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# plotly.tools.set_credentials_file(username='AlexanderBach', api_key='o4fx6i1MtEIJQxfWYvU1')


# For oversampling Library (Dealing with Imbalanced Datasets)
#from imblearn.over_sampling import SMOTE
from collections import Counter

# Other Libraries
import time


In [ ]:
%matplotlib inline
# import numpy as np
# import pandas as pd 
# import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
# import seaborn as sns
#import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
 

In [ ]:
#df = pd.read_csv('./LoanStats3a.csv')
df = pd.read_csv('./loan.csv')
df_origin=df

In [ ]:
df_description = pd.read_excel('./LCDataDictionary.xlsx').dropna()
df_description.style.set_properties(subset=['Description'], **{'width': '1000px'})

In [ ]:
#Inspecting the Null values
df.isnull().sum(axis=0)

In [ ]:
for col in ('dti_joint', 'annual_inc_joint', 'il_util', 'mths_since_rcnt_il', 'open_acc_6m', 'open_il_6m', 'open_il_12m',
           'open_il_24m', 'inq_last_12m', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl',
           'mths_since_last_record', 'mths_since_last_major_derog', 'mths_since_last_delinq', 'total_bal_il', 'tot_coll_amt',
           'tot_cur_bal', 'total_rev_hi_lim', 'revol_util', 'collections_12_mths_ex_med', 'open_acc', 'inq_last_6mths',
           'verification_status_joint', 'acc_now_delinq'):
    df[col] = df[col].fillna(0)

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
# id제거
df=df.iloc[:,2:] 

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
total_num = df.isnull().sum().sort_values(ascending=False)

perc = df.isnull().sum()/df.isnull().count() *100
perc1 = (round(perc,2).sort_values(ascending=False))

# Creating a data frame:
df_miss = pd.concat([total_num, perc1], axis =1 , keys =["Total Missing Values", "Percentage %"]).sort_values(by ="Percentage %", ascending = False)

In [ ]:
top_mis = df_miss[df_miss["Percentage %"]>0]
top_mis.reset_index(inplace=True)
top_mis

## Similar Distributions:
We will start by exploring the distribution of the loan amounts and see when did the loan amount issued increased significantly. 



### What we need to know:

Understand what amount was mostly issued to borrowers.
Which year issued the most loans.
The distribution of loan amounts is a multinomial distribution .

### Summary:

Most of the loans issued were in the range of 10,000 to 20,000 USD.
The year of 2015 was the year were most loans were issued.
Loans were issued in an incremental manner. (Possible due to a recovery in the U.S economy)
The loans applied by potential borrowers, the amount issued to the borrowers and the amount funded by investors are similarly distributed, meaning that Only controlled data set were released. If there is a loan value that is different from the loaner's demand, It will be able to predict the loan approval requirements of the company.


In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(121)
g = sns.distplot(df["loan_amnt"])
g.set_xlabel("", fontsize=12)
g.set_ylabel("Frequency Dist", fontsize=12)
g.set_title("Frequency Distribuition", fontsize=20)

plt.subplot(122)
g1 = sns.violinplot(y="loan_amnt", data=df, 
               inner="quartile", palette="hls")
g1.set_xlabel("", fontsize=12)
g1.set_ylabel("Amount Dist", fontsize=12)
g1.set_title("Amount Distribuition", fontsize=20)

plt.show()

#While the histogram method creates discontinuities by increasing the value of bin corresponding to each data in discrete, the KDE (Kernel Density Estimation) method adds each data as a function of the kernel, showing a smooth probability density function (pdf) as shown in the graph.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,5))

sns.distplot(df["loan_amnt"], ax=ax[0], color="#F7522F")
ax[0].set_title("Loan Applied by the Borrower", fontsize=14)
sns.distplot(df["funded_amnt"], ax=ax[1], color="#2F8FF7")
ax[1].set_title("Amount Funded by the Lender", fontsize=14)
sns.distplot(df["funded_amnt_inv"], ax=ax[2], color="#2EAD46")
ax[2].set_title("Total committed by Investors", fontsize=14)

In [ ]:
#year,month 분리
df['issue_month'], df['issue_year'] = df['issue_d'].str.split('-', 1).str

In [ ]:
months_order = ["Jan", "Feb", "Mar", "Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
df['issue_month'] = pd.Categorical(df['issue_month'],categories=months_order, ordered=True)

#Issue_d x loan_amount
plt.figure(figsize = (14,10))

g = sns.countplot(x='issue_month', 
                  data=df, 
                  hue='loan_status')
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set_xlabel("Months", fontsize=15)
g.set_ylabel("count", fontsize=15)
g.legend(loc='best')
g.set_title("Loan status by Months", fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize = (14,6))
#Looking the count of defaults though the issue_d that is The month which the loan was funded
g = sns.countplot(x='issue_year', data=df,
                  hue='loan_status')
g.set_xticklabels(g.get_xticklabels(),rotation=90)
g.set_xlabel("years", fontsize=15)
g.set_ylabel("Count", fontsize=15)
g.legend(loc='upper left')
g.set_title("Analysing Loan Status by Years", fontsize=20)
plt.show()

In [ ]:
# The year of 2015 was the year were the highest amount of loans were issued 
# This is an indication that the economy is quiet recovering itself.
plt.figure(figsize=(12,8))
sns.barplot(df['issue_year'], df['loan_amnt'], data=df, palette='tab10')
plt.title('Issuance of Loans', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Average loan amount issued', fontsize=14)

## Good Loans vs Bad Loans: Types of Loans:

### Types of Loans:
 In this section, we will see what is the amount of bad loans Lending Club has declared so far, of course we have to understand that there are still loans that are at a risk of defaulting in the future.
What we need to know:
The amount of bad loans could increment as the days pass by, since we still have a great amount of current loans.
Average annual income is an important key metric for finding possible opportunities of investments in a specific region.

### Summary:
Currently, bad loans consist 7.60% of total loans but remember that we still have current loans which have the risk of becoming bad loans. (So this percentage is subjected to possible changes.)
The NorthEast region seems to be the most attractive in term of funding loans to borrowers.
The SouthWest and West regions have experienced a slight increase in the "median income" in the past years.
Average interest rates have declined since 2012 but this might explain the increase in the volume of loans.
Employment Length tends to be greater in the regions of the SouthWest and West
Clients located in the regions of NorthEast and MidWest have not experienced a drastic increase in debt-to-income(dti) as compared to the other regions.

In [ ]:
df["loan_status"].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
ax = sns.countplot(y=df['loan_status'],order = df['loan_status'].value_counts().index)
ax = plt.title('Loan Status Distribution')

Loan Distribution by Status

Below are the definitions of each loan status:

Issued: New loan that has passed all LendingClub reviews, received full funding, and has been issued.

Current: Loan is up to date on all outstanding payments.

In Grace Period: Loan is past due but within the 15-day grace period.

Late (16-30): Loan has not been current for 16 to 30 days.

Late (31-120): Loan has not been current for 31 to 120 days.

Fully paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.

Default: Loan has not been current for an extended period of time.

Charged Off: Loan for which there is no longer a reasonable expectation of further payments. Upon Charge Off, the remaining 
             principal balance of the Note is deducted from the account balance.

More status definition : LendingClub

In [ ]:
df['loan_status'].value_counts()

In [ ]:
# Determining the loans that are bad from loan_status column

good_loan = ["Fully Paid", "Does not meet the credit policy. Status:Fully Paid"] 
unknown= ["Current" , "Issued"]          


df['loan_condition'] = np.nan

def loan_condition(status):
    if status in good_loan:
        return 'Good Loan'
    elif status in unknown:
        return 'Current'
    else:
        return 'Bad Loan'
    
    
df['loan_condition'] = df['loan_status'].apply(loan_condition)

In [ ]:
df['loan_condition'].value_counts()

In [ ]:
f, ax = plt.subplots(1,2, figsize=(18,8))

colors = ["#3ebfe2","#3791D7","#D72626" ]
labels = ["Current","Good Loans", "Bad Loans"]

plt.suptitle('Information on Loan Conditions', fontsize=18)

df["loan_condition"].value_counts().plot.pie(explode=[0,0,0.2], autopct='%1.2f%%', ax=ax[0], shadow=True, colors=colors, 
                                             labels=labels, fontsize=10, startangle=20)


# ax[0].set_title('State of Loan', fontsize=16)
ax[0].set_ylabel('% of Condition of Loans', fontsize=10)

# sns.countplot('loan_condition', data=df, ax=ax[1], palette=colors)
# ax[1].set_title('Condition of Loans', fontsize=20)
# ax[1].set_xticklabels(['Good', 'Bad'], rotation='horizontal')
palette = ["#3791D7","#D72626","#3ebfe2"]

sns.barplot(x=df['issue_year'], y=df['loan_amnt'], hue="loan_condition", data=df, palette=palette, estimator=lambda x: len(x) / len(df) * 100)
ax[1].set(ylabel="(%)")

## Loans Issued by Region and purpose
In this section we want to analyze loans issued by region in order to see region patters that will allow us to understand which region gives Lending Club.



## Summary: 
SouthEast , West and NorthEast regions had the high amount of loans issued while SouthWest and MidEast region has the lower.

The most common reason for a loan request is debt consolidation, followed by credit card and home improvement.

In [ ]:
df['addr_state'].unique()

# Make a list with each of the regions by state.

west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

df['region'] = np.nan

def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    


df['region'] = df['addr_state'].apply(finding_regions)
#df['region']

In [ ]:
# 위와 같은 df['region']결과값 다른코드

# regions_dict = pd.Series(df['region'].values,index=df['addr_state']).to_dict()
# df['region'] = df['addr_state'].map(regions_dict)
# df['region']

In [ ]:
# Create a new column 'region' by mapping the state value to region from the region dict


region_dist = df.groupby(['region']).size()
region_dist 
explode = (0, 0, 0, 0, 0) 
colors = ["#efc186","#a6ddb5","#dda19b","#9ebfe2","#3ebfe2"]
plt.figure(figsize=(15,6))
ax1 = plt.subplot(1, 2, 1)
ax1 = plt.pie(region_dist,autopct='%1.1f%%',explode=explode, labels=region_dist.index,colors = colors)
ax1 = plt.title('Loan Distribution By Region')
ax1 = plt.xlabel('Region')
ax1 = plt.axis('equal')  

ax2 = plt.subplot(1, 2, 2)
ax2 = sns.countplot(y=df['purpose'],order = df['purpose'].value_counts().index)
ax2 = plt.title('Loan Purpose Distribution')
ax2 = plt.ylabel('Loan Purposes')
ax2 = plt.xlabel('Frequency')
ax2 = plt.tight_layout()

In [ ]:
# We have 67429 loans categorized as bad loans
badloans_df = df.loc[df["loan_condition"] == "Bad Loan"]

# loan_status cross
loan_status_cross = pd.crosstab(badloans_df['region'], badloans_df['loan_status']).apply(lambda x: x/x.sum() * 100)
number_of_loanstatus = pd.crosstab(badloans_df['region'], badloans_df['loan_status'])


In [ ]:
charged_off = loan_status_cross['Charged Off'].values.tolist()
default = loan_status_cross['Default'].values.tolist()
not_meet_credit = loan_status_cross['Does not meet the credit policy. Status:Charged Off'].values.tolist()
grace_period = loan_status_cross['In Grace Period'].values.tolist()
short_pay = loan_status_cross['Late (16-30 days)'] .values.tolist()
long_pay = loan_status_cross['Late (31-120 days)'].values.tolist()



charged = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y= charged_off,
    name='Charged Off',
    marker=dict(
        color='rgb(192, 148, 246)'
    ),
    text = '%'
)

defaults = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y=default,
    name='Defaults',
    marker=dict(
        color='rgb(176, 26, 26)'
    ),
    text = '%'
)

credit_policy = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y= not_meet_credit,
    name='Does not meet Credit Policy',
    marker = dict(
        color='rgb(229, 121, 36)'
    ),
    text = '%'
)

grace = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y= grace_period,
    name='Grace Period',
    marker = dict(
        color='rgb(147, 147, 147)'
    ),
    text = '%'
)

short_pays = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y= short_pay,
    name='Late Payment (16-30 days)', 
    marker = dict(
        color='rgb(246, 157, 135)'
    ),
    text = '%'
)

long_pays = go.Bar(
    x=['MidWest', 'NorthEast', 'SouthEast', 'SouthWest', 'West'],
    y= long_pay,
    name='Late Payment (31-120 days)',
    marker = dict(
        color = 'rgb(238, 76, 73)'
        ),
    text = '%'
)




data = [charged, defaults, credit_policy, grace, short_pays, long_pays]
layout = go.Layout(
    barmode='stack',
    title = '% of Bad Loan Status by Region',
    xaxis=dict(title='US Regions')
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [ ]:
# Let's visualize how many loans were issued by creditscore
f, ((ax1, ax2)) = plt.subplots(1, 2)
cmap = plt.cm.coolwarm

by_credit_score = df.groupby(['issue_year', 'grade']).loan_amnt.mean()
by_credit_score.unstack().plot(legend=False, ax=ax1, figsize=(14, 4), colormap=cmap)
ax1.set_title('Loans issued by Credit Score', fontsize=14)
    
    
by_inc = df.groupby(['issue_year', 'grade']).int_rate.mean()
by_inc.unstack().plot(ax=ax2, figsize=(14, 4), colormap=cmap)
ax2.set_title('Interest Rates by Credit Score', fontsize=14)

ax2.legend(bbox_to_anchor=(-1.0, -0.3, 1.7, 0.1), loc=5, prop={'size':12},
           ncol=7, mode="expand", borderaxespad=0.)

In [ ]:
fig = plt.figure(figsize=(16,12))

ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(212)

cmap = plt.cm.coolwarm_r

loans_by_region = df.groupby(['grade', 'loan_condition']).size()
loans_by_region.unstack().plot(kind='bar', stacked=True, colormap=cmap, ax=ax1, grid=False)
ax1.set_title('Type of Loans by Grade', fontsize=14)


loans_by_grade = df.groupby(['sub_grade', 'loan_condition']).size()
loans_by_grade.unstack().plot(kind='bar', stacked=True, colormap=cmap, ax=ax2, grid=False)
ax2.set_title('Type of Loans by Sub-Grade', fontsize=14)

by_interest = df.groupby(['issue_year', 'loan_condition']).int_rate.mean()
by_interest.unstack().plot(ax=ax3, colormap=cmap)
ax3.set_title('Average Interest rate by Loan Condition', fontsize=14)
ax3.set_ylabel('Interest Rate (%)', fontsize=12)

In [ ]:
df['int_rate'].describe()

In [ ]:

# Average interest is 13.25% Anything above this will be considered of high risk let's see if this is true.
df['interest_payments'] = np.nan
lst = [df]

for col in lst:
    col.loc[col['int_rate'] <= 13, 'interest_payments'] = 'Low'
    col.loc[col['int_rate'] > 13, 'interest_payments'] = 'High'

In [ ]:
from scipy.stats import norm

plt.figure(figsize=(20,10))

palette = ['#009393', '#930000']
plt.subplot(221)
ax = sns.countplot(x='interest_payments', data=df, 
                  palette=palette, hue='loan_condition')

ax.set_title('The impact of interest rate \n on the condition of the loan', fontsize=14)
ax.set_xlabel('Level of Interest Payments', fontsize=12)
ax.set_ylabel('Count')

plt.subplot(222)
ax1 = sns.countplot(x='interest_payments', data=df, 
                   palette=palette, hue='term')

ax1.set_title('The impact of maturity date \n on interest rates', fontsize=14)
ax1.set_xlabel('Level of Interest Payments', fontsize=12)
ax1.set_ylabel('Count')


plt.subplot(212)
low = df['loan_amnt'].loc[df['interest_payments'] == 'Low'].values
high = df['loan_amnt'].loc[df['interest_payments'] == 'High'].values


ax2= sns.distplot(low, color='#009393', label='Low Interest Payments', fit=norm, fit_kws={"color":"#483d8b"}) # Dark Blue Norm Color
ax3 = sns.distplot(high, color='#930000', label='High Interest Payments', fit=norm, fit_kws={"color":"#c71585"}) #  Red Norm Color
plt.axis([0, 36000, 0, 0.00016])
plt.legend()


plt.show()

In [ ]:
# Drop columns with more than 80% of missing values
na_thresh = len(df)*80/100
df = df.dropna(thresh=na_thresh, axis=1)

# Check the new dimension
df.shape

In [ ]:
df.columns

In [ ]:
# blank -> na
df=df.replace('', np.nan, regex=True)

In [ ]:
#na 제거  55개 독립변수에서 34개됨.
#df.dropna(axis='index', how='any')
df=df.dropna(axis=0)
df

In [ ]:
df.isnull().sum(axis=0)

In [ ]:
n_miss_tot = df.isnull().sum().sort_values(ascending = False)
n_perc = df.isnull().sum()/df.isnull().count()*100
n_perc1 = (round(n_perc, 2).sort_values(ascending = False))
n_miss_df = pd.concat([n_miss_tot, n_perc1], axis = 1, keys=(["total", "Percentage"])).sort_values(
    by ="Percentage", ascending = False).head(8)
n_miss_df

In [ ]:
# lists number of unique values by each column
df.apply(lambda x: x.nunique())

In [ ]:
# removing the columns which has single(unique) values like - NA, 0, 'f', 'n' etc.
unique = df.nunique()
unique = unique[unique.values == 1]
unique


In [ ]:
# removing the columns which has single(unique) values like - NA, 0, 'f', 'n' etc.
df.drop(labels = list(unique.index), axis =1, inplace=True)

In [ ]:
# df['int_rate'].round(0).astype(int)

# plt.figure(figsize = (10,8))

# #Exploring the Int_rate
# plt.subplot(211)
# g = sns.distplot(np.log(df["int_rate"]))
# g.set_xlabel("", fontsize=12)
# g.set_ylabel("Distribuition", fontsize=12)
# g.set_title("Int Rate Log distribuition", fontsize=20)

# plt.subplot(212)
# g1 = sns.countplot(x=df['int_rate'].round(0).astype(int),data=df, 
#                    palette="Set2")
# g1.set_xlabel("Int Rate", fontsize=12)
# g1.set_ylabel("Count", fontsize=12)
# g1.set_title("Int Rate Distribuition", fontsize=20)

# plt.subplots_adjust(wspace = 0.2, hspace = 0.6,top = 0.9)

# plt.show()

In [ ]:
import plotly.offline as py 
import plotly.figure_factory as ff 
import plotly.graph_objs as go 
from plotly import tools 
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot 
py.init_notebook_mode(connected= True)


data = []
for i in df["loan_status"].unique():
    data.append(go.Box(y = df[df["loan_status"]==i]["int_rate"], name = i))
    
layout = go.Layout(title = 'Interested Rate based on Loan Status', 
                   xaxis = dict(title = 'Status'), 
                   yaxis = dict(title = 'Interest Rate'))
fig = dict(data = data, layout = layout)
py.iplot(fig)

In [ ]:
data1= []
for i in df["grade"].unique():
    data1.append(go.Box(y = df[df["grade"]==i]["int_rate"], name = i))
    
layout = go.Layout(title = 'Interested Rate based on Grades', 
                   xaxis = dict(title = 'Grades'), 
                   yaxis = dict(title = 'Interest Rate'))
fig = dict(data = data1, layout = layout)
py.iplot(fig)

In [ ]:
df.columns

In [ ]:
data2= []
for i in df["emp_length"].unique():
    data2.append(go.Box(y = df[df["emp_length"]==i]["int_rate"], name = i))
    
layout = go.Layout(title = 'int_rate on emp_length', 
                   xaxis = dict(title = 'emp_length'), 
                   yaxis = dict(title = 'int_rate'))
fig = dict(data = data2, layout = layout)
py.iplot(fig)

In [ ]:
data3= []
for i in df["purpose"].unique():
    data3.append(go.Box(y = df[df["purpose"]==i]['funded_amnt'], name = i))
    
layout = go.Layout(title = 'funded_amnt on purpose', 
                   xaxis = dict(title = 'purpose'), 
                   yaxis = dict(title = 'funded_amnt'))
fig = dict(data = data3, layout = layout)
py.iplot(fig)

In [ ]:
# # percentage to numeric
# df['int_rate'] = pd.to_numeric(df['int_rate'].str.strip('%'),errors='coerce')
# #df['int_rate'][:]

In [ ]:
#df['int_rate']로 credit_score종속변수 만들기

df['credit_score']=round(np.reciprocal(df['int_rate'])*5000)



In [ ]:
df[["sub_grade","int_rate","credit_score"]]

In [ ]:
plt.figure(figsize=(10, 8)) 
plt.figure(1)
corr = df.corr()
sns.heatmap(corr)

#### Data cleaning

1.데이터 na 값,즉 결측치가 80%이상인 행 지운다

2.데이터를 continuous data(float,int) 와 categorical data로 나눈다

3.연속형데이터:

  1)상관관계 분석: 다중공선성 제거.상호 상관관계 높은 변수 통합(통합변수 만들거나 pca등으로 차원감소)하거나 삭제.
    연속형변수의 분산확인: 분산이 0에 가까운 변수 삭제
    
  2)결측치는 중앙값이나 평균등 논리적 의미 있는 값으로 대체.(의미 찾기어려우면 클러스터링후 대표값)
   이상치는 IQR이나 왜도를 측정하여 이상치 체크하여 이상치 제거할수 있으나 이상치의 숨은 의미를 찾는것이 중요.
   예를들어 연봉이 높은 이상치일 경우 전문직일수 있으니 전문직 유무의 변수를 만드는것이 더 정확하다. 그러므로 이상치도 
   최대한 유지하며 제거없이 스케일 표준화가 낫다. 결측치 대체를 못한 행은 지운다.
   
4.카데고리형:

  범주의 개수(number of unique values)가 1개인 변수는 지운다. 아무런 분포가 없으므로 영향을 안준다
  유니크(number of unique values)값이 너무 많고 분석에 필요없는 데이터 지운다(예를 들어 zipcode 나 url 주소)
  카테고라이즈 해서 분석해야 할것 즉, date, region, job title 같은  변수 따로 뺴서 분석.
  신용상태 분류나 신용등급 회귀분석에는 필요없어서 따로 분석하기 위해 따로 빼둔다.
  결측치는 Missing 값으로 imputation.
  각 범주형 변수에서 unique values를 그룹화하여 종속변수와의 평균값 order순으로 인코딩한다.(카데고리형도 순서형 변수로 변환)
  one-hot-encoding은 차원이 높아지므로 비선호.


###### 종속변수와 상관관계 높은 변수선택이나 낮은 변수 제거 그리고  종속변수와 직접적 연관있는 변수제거, 카테고리형 변수를 종속변수와의 평균값 order순으로 인코딩은 모델 설정전에 따로 해야한다. 분석할 목적과 모델에 따라 종속변수가 변하기 때문이다.


####  Feature selection
1.연속형변수: 종속변수와 상관관계 높은변수 선택(연속형은 피어슨 계수,순위형은 스피어만 계수) -modeling 하기전

2.카테고리형 변수:인코딩으로 순위형으로 바꾸면 연속형변수처럼 처리. 분류나 회귀분석에 맞지않으면 제거.변수별 시각화 분석으로만 사용.

In [ ]:
# object 타입변수들 unique 개수 봐서 필요없는것 삭제
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [ ]:
# 분류나 회귀분석에 필요없는 변수 제거 (object 타입변수들 unique 개수 많은것+ 날짜변수)
df.drop(['emp_title','url','title','zip_code','issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d'], inplace=True, axis=1, errors='ignore')
# 위에서 분석을 위해 만든 변수들 이제 예측분석에 필요없으니 제거
df.drop(['issue_year','issue_month','region','interest_payments'], inplace=True, axis=1, errors='ignore')

In [ ]:
# heatmap 보고 상호 상관관계 높은 변수 제거
df.drop(['loan_amnt', 'funded_amnt_inv','int_rate','installment','total_acc','out_prncp_inv','total_pymnt_inv', 'total_rec_prncp', 'total_rec_int','collection_recovery_fee','acc_now_delinq','pub_rec_bankruptcies','last_pymnt_amnt','total_rev_hi_lim','tax_liens','sub_grade'], inplace=True, axis=1, errors='ignore')
df.columns

In [ ]:
# object 타입변수들 unique 개수 많은것+ 날짜변수 삭제후
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

## Classification

##### preprocessing (종속변수- loan_condition)

In [ ]:
# 분산이 0에 가까운 변수들 ( feature selecting 할때 사용한다. R에서 nearZeroVar 함수역할)
# tot_coll_amt, collection_12_mths_ex_med, recoveries,total_rec_late_fee,revol_bal,pub_rec

df=df.drop(['collections_12_mths_ex_med','tot_coll_amt'],axis = 1)   
# recoveries,total_rec_late_fee를 지워야 하지만 bad loan을 나타내는 중요변수기에 삭제전에 영향을 보고 뒤에서 지울것이다.

# 상관성이 똑같은 credit score이 있어서 삭제.
df=df.drop('grade', axis = 1)

In [ ]:
df.head(1)

In [ ]:
# 개별 변수를 인코딩 한다. 여기선 loan_condition이 문자변수라 숫자화 하기위해 loan_condition_E로 인코딩함.
from scipy.stats import trim_mean
# m = stats.trim_mean(train[''], 0.1) # Trim 10% at both ends of the mean 극단치 제외하기위해 mean 대신 trim_mean 사용

def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique() #각 특성의 카테고리값을 val에 넣었다
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'credit_score']].groupby(feature)['credit_score'].apply(trim_mean, 0.1) #각 특성의 카테고리값의 credit_score 평균을 spmean에 넣는다
    ordering = ordering.sort_values('spmean') # credit_score 평균값이 작은 카테고리부터 sort된다.
    ordering['ordering'] = range(1, ordering.shape[0]+1) # 1부터 카데고리수만큼 순서를 정한후
    ordering = ordering['ordering'].to_dict()

    for cat, o in ordering.items(): # 키,밸류 뽑아서
        frame.loc[frame[feature] == cat, feature+'_E'] = o 
        
encode(df, 'loan_condition')

In [ ]:
# 상관성이 똑같은 loan_condition_E이 있어서 삭제.
df=df.drop('loan_condition', axis = 1)

In [ ]:
df= df.drop('loan_status', axis = 1) # 종속변수 'loan_condition' 와 직접적인 상관관계가 있어 제외한다.
df.head(1)

In [ ]:
quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
qualitative = [f for f in df.columns if df.dtypes[f] == 'object']


In [ ]:
# missing categories imoutation
for c in qualitative:
    df[c] = df[c].astype('category')
    if df[c].isnull().any():
        df[c] = df[c].cat.add_categories(['MISSING'])
        df[c] = df[c].fillna('MISSING')
        # null -> missing 카데고리를 만든다

In [ ]:
# 'loan_condition_E' 에서 good loan, bad loan을 분류한다.  loan_condition_E를 종속변수로 다시 변수 셀렉션을 해야한다.

In [ ]:
df['loan_condition_E'].value_counts()

# order: bad(1)-good(2)-current(3)

In [ ]:
df=df[df['loan_condition_E']!=3]

In [ ]:
# Encording

from scipy.stats import trim_mean
# m = stats.trim_mean(train[''], 0.1) # Trim 10% at both ends of the mean 극단치 제외하기위해 mean 대신 trim_mean 사용
    
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique() #각 특성의 카테고리값을 val에 넣었다
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'loan_condition_E']].groupby(feature)['loan_condition_E'].apply(trim_mean, 0.1) #각 특성의 카테고리값의 credit_score 평균을 spmean에 넣는다
    ordering = ordering.sort_values('spmean') # credit_score 평균값이 작은 카테고리부터 sort된다.
    ordering['ordering'] = range(1, ordering.shape[0]+1) # 1부터 카데고리수만큼 순서를 정한후
    ordering = ordering['ordering'].to_dict() #딕셔너리 키밸류 셋으로 만든후
    
    for cat, o in ordering.items(): # 키,밸류 뽑아서
        frame.loc[frame[feature] == cat, feature+'_E'] = o # 카테고리 이름과 변수 이름이 만나는 행렬의 밸류값 즉 credit_score평균값을 꺼낸다.
    
qual_encoded_C = []
for q in qualitative:  
    encode(df, q)
    qual_encoded_C.append(q+'_E')
print(qual_encoded_C)

In [ ]:
# 인코딩한 데이터로 대체해서 df 새로 만듬
df_C=df[quantitative+qual_encoded_C]

In [ ]:
#df_C['loan_condition_E'].value_counts()

# order: bad(1)-good(2)-current(3)

In [ ]:
#df_C=df_C[df_C['loan_condition_E']!=3]

In [ ]:
#df_C= df_C.drop('loan_status_E', axis = 1) # 종속변수 'loan_condition_E' 와 직접적인 상관관계가 있어 제외한다.

In [ ]:
df_C.columns

In [ ]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['loan_condition_E'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
    
features = quantitative + qual_encoded_C
spearman(df_C, features)

In [ ]:
df_C.drop(['open_acc_6m','open_il_12m','open_il_24m','annual_inc_joint', 
'dti_joint','il_util','open_rv_24m','all_util','inq_last_12m','inq_fi'], inplace=True, axis=1)


In [ ]:
df_C.columns  

In [ ]:
#변수 그룹별 상관관계
#Correlation matrix

plt.figure(figsize=(10, 8)) 
plt.figure(1)
corr = df_C[quantitative].corr()
sns.heatmap(corr)

plt.figure(figsize=(10, 8))
plt.figure(2)
corr = df_C[qual_encoded_C+['loan_condition_E']].corr()
sns.heatmap(corr)

plt.figure(figsize=(10, 8))
plt.figure(3)
corr = pd.DataFrame(np.zeros([len(quantitative), len(qual_encoded_C)+1]), index=quantitative, columns=qual_encoded_C+['loan_condition_E'])
for q1 in quantitative:
    for q2 in qual_encoded_C:
        corr.loc[q1, q2] = df_C[q1].corr(df_C[q2])
sns.heatmap(corr)

In [ ]:
df_C.tot_cur_bal.corr(df_C.home_ownership_E)
# 3번째 히트맵. 상관관계 큰문제 없고 중요변수 아니니 넘어간다.

In [ ]:
df_C.funded_amnt.corr(df_C.total_pymnt)
#1번째 히트맵. 높은 상관 관계이고 중요변수이면서 논리적으로도 높은 상관관계이기에 대표변수를 사용해야 한다. 대표변수는 뒤에서 정할것이다. 

###### bad loan을 결정해주는 대표변수지만 분산이 없어서 지워야할 변수를 보자.  상관관계가 적다. 미래고객의 대출 상환력을 예측하는 모델에선 둘다 지우고 현재 current 상태의 고객을 예측하기 위해선 중요변수이므로 둘다 사용해야 한다. 여기선 미래고객의 예측하기위한 모델이므로 뒤에서 둘다 제거한다.

In [ ]:
df_C.recoveries.corr(df_C.out_prncp)


In [ ]:
def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
f = pd.melt(df, id_vars=['loan_condition_E'], value_vars=qualitative)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(boxplot, "value", "loan_condition_E")

In [ ]:
# pairplot 에서 loan_condition_E값으로 melt 하기 위해서는 겹쳐지니 뺸다.
quantitative.remove('loan_condition_E')

In [ ]:
def pairplot(x, y, **kwargs):
    ax = plt.gca()
    ts = pd.DataFrame({'time': x, 'val': y})
    ts = ts.groupby('time').mean()
    ts.plot(ax=ax)
    plt.xticks(rotation=90)
    
f = pd.melt(df_C, id_vars=['loan_condition_E'], value_vars=quantitative+qual_encoded_C)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False, size=5)
g = g.map(pairplot, "value", "loan_condition_E")

###### recoveries, out_prncp 는 bad loan(loan_condition=1)만 존재하기에 강력한 분류 변수이다. 하지만 인과가 바뀌었으니 지운다.

In [ ]:
df_g=df_C[['recoveries','out_prncp','loan_condition_E']]

In [ ]:
df_g.groupby(df_g.loan_condition_E).describe()

###### 위에서 분산으로 봤을때 지워야 하지만 여기서recoveries ,out_prncp 는 bad loan(loan_condition=1)만 존재하기에 강력한 분류 변수이다. 하지만 recoveries는 인과가 바뀌었으니 지워야 하고 현재current 상태의 고객의 미래상환을 예측하려면 out_prncp는 강력한 bad loan 결정변수가 된다.

In [ ]:
# 이번 모델에선 미래고객의 상환력을 예측하기에 둘다 지운다
df_C.drop(['recoveries','out_prncp'], inplace=True, axis=1)

##### 여기서 중요한 점은 모델의 목적이다. 미래고객의 대출상환력을 예측하려면 'recoveries','out_prncp' 두변수 모두 지워야 한다

In [ ]:
df_p=df_C[['total_pymnt','funded_amnt','loan_condition_E']]
#df_p.sort_values(by='loan_condition_E')
df_p.groupby(df_p.loan_condition_E).mean()


##### loan 상태가 좋을수록(2.0) 두 집단의 funded_amnt는 비슷해도 total_pymnt 가 높다. 즉 돈을 빌려도 잘갚는다. 집단2.0을 보면 total_pymnt >unded_amnt 임을 볼수 있다. 즉 빌린돈보다 갚은돈이 많다. 이자까지 잘 갚는것을 볼수있다. 그러나 집단1.0은 절반도 갚지 못하는걸 볼수있다

In [ ]:
# 'funded_amnt','total_pymnt' 는 대출상태를 구분짓는 강력한 분류 변수이다. 하지만 인과가 바뀌었으니 지운다. 대출상태가 나빠서 돈을 안갚은게 아니라 돈을 안갚아서 대출상태가 나빠진것
df_C.drop(['total_pymnt'], inplace=True, axis=1)

###### funded_amnt, total_pymnt 는 앞에서 높은 상관관계가 있는 변수이면서 대출상태를 구분짓는 강력한 분류 변수이다. 하지만  total_pymnt는 인과가 바뀌었으니 지운다.  대출상태가 나빠서 돈을 안갚은게 아니라 돈을 안갚아서 대출상태가 나빠진것 이기 때문이다. funded_amnt만 봐서는 돈을 빌리는 양만으론 대출갚는 유무와 인과관계가 없다.

In [ ]:
df_C.columns

In [ ]:
# Extract features and labels 

Label = df_C['loan_condition_E']
features = df_C.drop('loan_condition_E', axis = 1)

In [ ]:
# Training and Testing Sets
from sklearn.model_selection import train_test_split

# 데이터를 훈련+검증 세트 그리고 테스트 세트로 분할
X_trainval, X_test, y_trainval, y_test = train_test_split(
    features, Label, random_state=0)
# 훈련+검증 세트를 훈련 세트와 검증 세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X_trainval, y_trainval, random_state=1)
print("train 세트의 크기: {}   validate 세트의 크기: {}   test 세트의 크기:"
      " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))


# X_train, y_train  //  X_valid, y_valid  //  X_test, y_test

#### 데이터 전처리 - 데이터 표준화 작업 (scaling)

In [ ]:
### 데이터 전처리 - 데이터 표준화 작업 (scaling)
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler  # 0~1 사이의 숫자로 바꾼다. 양수가 필요할때 변환

sc = RobustScaler()
transformer = sc.fit(X_train)
transformer

sc2= StandardScaler()
transformer2 = sc2.fit(X_train)

sc3= MinMaxScaler()
transformer3 = sc3.fit(X_train)

# 4분위수를 이용한 표준화 스케일 방법인 RobustScaler를 이용.이상치 영향 약화. 


주의할 점은 표본평균과 표본표준편차를 계산할 때 test set, validation set을 사용하면 안된다는 것이다. 표본평균/표준편차는 training set으로만 계산해야한다. Training set에서 계산한 값들로 validation set, test set을 “표준화“해주면 된다.

In [ ]:
#위에서 training set으로 fit 시킨 평균,분산 값을 이용하여 각 세트의 독립변수(X)들을 변환시킨다. 
X_train_std=transformer.transform(X_train)
X_valid_std=transformer.transform(X_valid)
X_test_std=transformer.transform(X_test)


X_train_std2=transformer2.transform(X_train)
X_valid_std2=transformer2.transform(X_valid)
X_test_std2=transformer2.transform(X_test)

X_train_std3=transformer3.transform(X_train)
X_valid_std3=transformer3.transform(X_valid)
X_test_std3=transformer3.transform(X_test)


### Model  (이번 모델의 목적은 미래고객의 대출상환 정도를 예측 분류 하는것이다)

https://github.com/ExcelsiorCJH/Hands-On-ML/blob/master/Chap07-Ensemble_Learning_and_Random_Forests/Chap07-Ensemble_Learning_and_Random_Forests.ipynb 참고

##### RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=42)
forest.fit(X_train_std, y_train)

In [ ]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(forest.score(X_valid_std, y_valid)))

In [ ]:
# Output feature importance coefficients, map them to their feature name, and sort values
coef = pd.Series(forest.feature_importances_, index = X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 5))
coef.head(25).plot(kind='bar')
plt.title('Feature Significance')
plt.tight_layout()

훈련 세트의 정확도와 테스트의 정확도가 차이가 나는 것을 볼때 오버피팅이다. 과대적합을 막기 위해서 모델튜닝을 할수도 있지만 앙상블을 적용해보자

##### 그래디언트 부스팅
부스팅의 인기 알고리즘인 그래디언트 부스팅(Gradient Boosting)에 대해 알아보도록 하자. 그래디언트 부스팅은 '아다부스트' 처럼 전의 학습된 모델의 오차를 보완하는 방향으로 모델(분류기, 학습기)을 추가해주는 방법은 동일하다. 하지만, 그래디언트 부스팅은 아다부스트 처럼 학습단계 마다 데이터 샘플의 가중치를 업데이트 해주는 것이 아니라 학습 전단계 모델에서의 잔여 오차(residual error)에 대해 새로운 모델을 학습시키는 방법이다.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state=42)
gbrt.fit(X_train_std, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_valid_std, y_valid)))

모델 튜닝으로 오버피팅 막기위해 트리의 최대 깊이를 줄여 사전 가지치기를 강하게 하거나 학습률을 낮출 수 있습니다.

In [ ]:
gbrt = GradientBoostingClassifier(random_state=42, max_depth=2)
gbrt.fit(X_train_std, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_valid_std, y_valid)))

In [ ]:
gbrt = GradientBoostingClassifier(random_state=42, learning_rate=0.2)
gbrt.fit(X_train_std, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_valid_std, y_valid)))


In [ ]:
gbrt = GradientBoostingClassifier(random_state=42, learning_rate=0.3,max_depth=3)
gbrt.fit(X_train_std, y_train)

print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_valid_std, y_valid)))

In [ ]:
gbrt = GradientBoostingClassifier(random_state=42) #best model 선택
gbrt.fit(X_train_std, y_train)

# Output feature importance coefficients, map them to their feature name, and sort values
coef = pd.Series(gbrt.feature_importances_, index = X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 5))
coef.head(25).plot(kind='bar')
plt.title('Feature Significance')
plt.tight_layout()

In [ ]:
# 최종 테스트 (위에서 가장 성능좋은것 실행후 하자 이름이 모두 gbrt로 같기 때문.)
print("훈련 세트 정확도: {:.3f}".format(gbrt.score(X_train_std, y_train)))
print("테스트 세트 정확도: {:.3f}".format(gbrt.score(X_test_std, y_test)))

#### 검증시보다 테스트시에 정확도가 더 오른걸 볼수있다. train,validation,test set 모두에서 비슷한 성능을 내고 있다. bias,variance 둘다 낮은 최적의 에러율을 보인다.

In [ ]:
# Get numerical feature importances

feature_list = list(X_train.columns)
importances = list(gbrt.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');


In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 80% of importance retained
plt.hlines(y = 0.80, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

##### 주요변수 2개가 67%, 4개가 81%를 설명하는 것을 볼수있다

#### 주요 변수를 바탕으로 loan_condition 에 끼치는 영향을 본다

In [ ]:
df_C.columns

In [ ]:
df_p=df_C[['credit_score','total_rec_late_fee','dti','term_E','annual_inc','funded_amnt','loan_condition_E']]
df_p.head()

In [ ]:
#df_p.sort_values(by='loan_condition_E')
df_p.groupby(df_p.loan_condition_E).mean()


#### 주요변수의 영향을 보면 credit_score가 대출 상환에 가장 큰 영향을 보여준다. 역시 신용이 높을수록(이자가 낮을수록) 대출상환이 좋다. 그리고 납부한 연체료가 낮을수록, dti(debt/income) 가 낮을수록, 대출기간이 짧을수록,연 수익이 높을수록,빌린 금액이 작을수록 대출상환력이 높다. 특히 신용등급과 연체정도 두개 변수가 약 70%정도 모델을 설명하고 있는것을 볼수있다. 최신 current,issued 된 대출상황을 보면 특히 신용등급이 많이 높아진것을 볼수있는데 LC에서 자체적으로 데이터분석을 하였을때 같은 결론을 내리고 대출조건에 신용등급조건을 높인것을 예측할수 있다.  

In [ ]:
#df['term'].value_counts()

In [ ]:
#df_p['term_E'].value_counts()

In [ ]:
#df_p.describe()

##### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

gbrt.fit(X_train_std, y_train)
y_pred = gbrt.predict(X_test_std)
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
(cm[0,0]+cm[1,1])/np.sum(cm)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

#Confusion matrix에서 구한 값과 같다

##### Precision, recall and f1-score

In [ ]:

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

Accuracy 정확도
전체 샘플 중 맞게 예측한 샘플 수의 비율
모형 트레이닝 즉 최적화에서 목적함수로 사용

accuracy=TP+TN / TP+TN+FP+FN
 
Precision 정밀도
Positive 클래스에 속한다고 출력한 샘플 중 실제로 Positive 클래스에 속하는 샘플 수의 비율

precision=TP / TP+FP
 
Recall 재현율
실제 Positive 클래스에 속한 샘플 중에 Positive 클래스에 속한다고 출력한 표본의 수

TPR(true positive rate)
sensitivity(민감도)
recall=TP / TP+FN
 
Fall-Out 위양성율
실제 Positive 클래스에 속하지 않는 샘플 중에 Positive 클래스에 속한다고 출력한 표본의 수

FPR(false positive rate)
specificity(특이도) = 1 - fall-out
fallout=FP / FP+TN
 
F (beta) score
정밀도(Precision)과 재현율(Recall)의 가중 조화 평균
Fβ=(1+β2)(precision×recall)/(β2.precision+recall)
 
F1 score
beta = 1
F1=2⋅precision⋅recall/(precision+recall)

결과를 볼때 1번집단(bad loan)을 잘 판별하지 못하였다. 그이유는 support에서도 알수 있지만 샘플의 수가 상대적으로 적고 훈련 셋을 비교하면 더욱 1번군의 샘플의 수가 적다. 2번 샘플의 양이 3배이상 많기 때문에 좋은 정확도를 보이고 있다. 그리고 

##### ROC curve

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_curve

y_pred_proba = gbrt.predict_proba(X_valid_std)[::,1]
fpr, tpr, _ = roc_curve(y_valid, y_pred_proba, pos_label=2) 
# fpr, tpr, _ = metrics.roc_curve(y_valid,  y_pred_proba)
auc = metrics.roc_auc_score(y_valid, y_pred_proba)
plt.plot(fpr,tpr,label="ROC Curve, auc="+str(auc))
plt.legend(loc=4)
plt.show()




Even if your data is binary, but the labels are '1' and '2', it will throw this error. So according to the documentation for the roc_curve() function from scikit-learn, you need to specify exactly which string label to use as the "positive class". So if your labels were 'T' and 'F' in your y_validate variable, you would do: fpr, tpr, _ = roc_curve(y_validate, status[:,1], pos_label=2). if labels are 'T' and 'F', pos_label='T'. In my case, pos_label is float so don't need ''.

In [ ]:
from sklearn.metrics import roc_curve
    
y_pred_prob = gbrt.predict_proba(X_valid_std)[:,1]

fpr, tpr, thresholds = roc_curve(y_valid, y_pred_prob, pos_label=2)
auc = metrics.roc_auc_score(y_valid, y_pred_prob)

# create plot
plt.plot(fpr, tpr, label='ROC curve,  auc='+str(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
_ = plt.xlabel('False Positive Rate')
_ = plt.ylabel('True Positive Rate')
_ = plt.title('ROC Curve')
_ = plt.xlim([-0.02, 1])
_ = plt.ylim([0, 1.02])
_ = plt.legend(loc="lower right")


# save figure
#plt.savefig('roc_curve.png', dpi=200)

AUC (Area Under the Curve)
AUC는 ROC curve의 면적을 뜻한다. Fall-Out 대비 Recall 값이 클 수록 AUC가 1에 가까운 값이며 민감한 모형이다.

###### Precision-recall curve

Precision Recall Plot (이하 PR 그래프)의 경우도 ROC 와 유사한데, 주로 데이타 라벨의 분포가 심하게 불균등 할때 사용한데, 예를 들어 이상 거래 검출 시나리오의 경우 정상 거래의 비율이 비정상 거래에 비해서 압도적으로 많기 때문에 (98%, 2%) 이런 경우에는 ROC 그래프보다 PR 그래프가 분석에 더 유리하다.

PR 그래프는 X 축을 Recall 값을, Y축을 Precision 값을 사용한다.


Sensitive (Recall) = (TP) / P

Precision = TP / (TP+FP)


In [ ]:
# from sklearn.metrics import precision_recall_curve 
# from sklearn.metrics import average_precision_score

# precision_score= average_precision_score(y_valid, y_pred_prob,pos_label=2)
# precision, recall, thresholds = precision_recall_curve(y_valid, y_pred_prob, pos_label=2)

# # create plot
# plt.plot(precision, recall, label='Precision-recall curve,  AP='+str(precision_score))
# _ = plt.xlabel('Recall')
# _ = plt.ylabel('Precision')
# _ = plt.title('Precision-recall curve')
# _ = plt.xlim([0.735, 1])
# _ = plt.ylim([0, 1.02])
# _ = plt.legend(loc="lower left")

# # save figure
# plt.savefig('precision_recall.png', dpi=200)

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_valid, y_pred_prob,pos_label=2)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature

precision, recall, _ = precision_recall_curve(y_valid, y_pred_prob, pos_label=2)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
from sklearn.metrics import average_precision_score

average_precision_score(y_valid, y_pred_prob,pos_label=2)

성능판단은 ROC 그래프의 경우에는 Y=X 그래프를 기준으로 그래프 윗쪽에 있는 경우 쓸만한 모델로 판단을 했는데, PR 그래프의 경우 Base line이라는 것을 사용한다. 

Base line = P / (P+N) 으로 정하는데, P는 데이타에서 Positive 레이블의 수, N 은 전체 데이타의 수이다. 예를 들어 암 데이타에서 암 양성이 300개 이고, 전체 데이타가 700이면 Base line은 300/(700+300) = 0.3 이 된다.  

이경우는 Base line= 36610 / 12949 + 36610 = 0.738 이다. 


######  Implement Naive Bayes 

In [ ]:
# Naive Bayes는 X_train 값이 양수(+)여야 해서 MinMaxScaler를 사용하여 스케일링 하였다.
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_std3, y_train)
y_pred = nb.predict(X_valid_std3)
print(accuracy_score(y_valid, y_pred))

##### Scikit-Learn의 VotingClassifier를 이용해 앙상블 학습하기

https://github.com/ExcelsiorCJH/Hands-On-ML/blob/master/Chap07-Ensemble_Learning_and_Random_Forests/Chap07-Ensemble_Learning_and_Random_Forests.ipynb 참고

##### Hard voting
VotingClassifier에서 voting='hard' 인 경우 각 분류기의 예측값(레이블)을 가지고 다수결 투표를 통해 최종 앙상블 예측이 이루어진다.

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import VotingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.naive_bayes import MultinomialNB

# log_clf = LogisticRegression(random_state=42)
# rnd_clf = RandomForestClassifier(random_state=42)
# svm_clf = SVC(random_state=42)
# nb_clf = MultinomialNB()

# voting_clf = VotingClassifier(
#     estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf),('nb', nb_clf)],
#     voting='hard')
# voting_clf.fit(X_train_std, y_train)

In [ ]:
# SVC 가 시간이 많이 걸리는 관계로 제외후 계산, NB는 스케일링이 다르므로 제외. 대신 gredient boosting 사용
 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
gbrt_clf = GradientBoostingClassifier(random_state=42) 

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf),('gb', gbrt_clf)],
    voting='hard')
voting_clf.fit(X_train_std, y_train)

In [ ]:
# # LabelEncoder 버그로 인해 생기는 DeprecationWarning 막기
# # https://github.com/scikit-learn/scikit-learn/pull/9816
# # https://stackoverflow.com/questions/49545947/sklearn-deprecationwarning-truth-value-of-an-array
# import warnings
# warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
# from sklearn.metrics import accuracy_score

# for clf in (log_clf, rnd_clf, svm_clf, nb_clf, voting_clf):
#     clf.fit(X_train_std, y_train)
#     y_pred = clf.predict(X_valid_std)
#     print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, gbrt_clf, voting_clf):
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_valid_std)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

##### Soft voting
VotingClassifier에서 voting='soft' 인 경우 각 분류기의 예측값(레이블)의 확률을 가지고 평균을 구한 뒤, 평균이 가장 높은 클래스로 최종 앙상블 예측이 이루어진다. 이러한 방법을 간접 투표(soft voting)이라 한다.

In [ ]:
# log_clf = LogisticRegression(random_state=42)
# rnd_clf = RandomForestClassifier(random_state=42)
# svm_clf = SVC(probability=True, random_state=42)
# nb_clf= MultinomialNB()

# voting_clf = VotingClassifier(
#     estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf), ('nb', nb_clf)],
#     voting='soft')
# voting_clf.fit(X_train_std, y_train)

In [ ]:
# from sklearn.metrics import accuracy_score

# for clf in (log_clf, rnd_clf, svm_clf, nb_clf, voting_clf):
#     clf.fit(X_train_std, y_train)
#     y_pred = clf.predict(X_valid_std)
#     print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

In [ ]:
# SVC 가 시간이 많이 걸리는 관계로 제외후 계산, NB는 스케일링이 다르므로 제외. 대신 gredient boosting 사용
 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
gbrt_clf = GradientBoostingClassifier(random_state=42) 

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf),('gb', gbrt_clf)],
    voting='soft')
# voting_clf.fit(X_train_std, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, gbrt_clf, voting_clf):
    clf.fit(X_train_std, y_train)
    y_pred = clf.predict(X_valid_std)
    print(clf.__class__.__name__, accuracy_score(y_valid, y_pred))

##### 배깅과 페이스팅 (Bagging and Pasting)
위의 '투표기반 분류기'에서는 여러개의 머신러닝 알고리즘을 이용해 분류기를 만들어 앙상블 학습을 했다. 이번에 알아볼 또 다른 방법으로는 하나의 알고리즘을 사용하지만 학습 데이터셋을 랜덤하게 추출하여 모델(분류기)을 각각 다르게 학습시키는 방법이다.

이렇게, 학습 데이터셋에서 랜덤하게 추출할 때, 중복을 허용하는 방식을 배깅(bagging)이라고 한다. 배깅(bagging)은 bootstrap aggregating의 줄임말이며 통계학에서는 중복을 허용한 리샘플링(resampling)을 부트스트래핑(bootstraping)이라고 한다. 이와 반대로 중복을 허용하지 않는 샘플링 방식을 페이스팅(pasting)이라고 한다.

각 모델이 학습된 후에 새로운 데이터에 대해서는 '투표기반 분류기'와 동일하게 분류(classification)일 때는 최빈값(mode) 즉, 가장 많은 예측 클래스로 앙상블이 예측하며, 회귀(regression)일 경우에는 각 분류기의 예측값의 평균을 계산하여 평균값을 예측값으로 한다.

각 모델은 전체 학습 데이터셋으로 학습시킨 것보다 편향되어 있지만, 앙상블을 통해 편향(bias)과 분산(variance)이 감소한다. 일반적으로 앙상블 학습은 전체 학습 데이터셋을 이용해 하나의 모델을 학습시킬 때와 비교해서 편향은 비슷하지만 분산은 줄어든다고 한다.

Scikit-Learn은 배깅과 페이스팅을 간편하게 사용할 수 있도록 분류일 경우에는 BaggingClassifier를 회귀일 경우 BaggingRegressor를 제공한다.

아래는 BaggingClassifier를 이용해 500개의 의사결정나무(decision tree)모델을 만들어 배깅을 적용한 앙상블 학습을 한 예제이다.


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train_std, y_train)
y_pred = bag_clf.predict(X_valid_std)

print('Accuracy =', accuracy_score(y_valid, y_pred))

##### OOB(Out-of-Bag) 평가
배깅은 중복을 허용하는 리샘플링(resampling) 즉, 부트스트래핑(bootstraping) 방식이기 때문에 전체 학습 데이터셋에서 어떠한 데이터 샘플은 여러번 샘플링 되고, 또 어떠한 샘플은 전혀 샘플링 되지 않을 수가 있다. 평균적으로 학습 단계에서 전체 학습 데이터셋 중 63% 정도만 샘플링 되며, 샘플링 되지 않은 나머지 37% 데이터 샘플들을 oob(out-of-bag) 샘플이라고 한다.

앙상블(배깅) 모델의 학습 단계에서는 oob 샘플이 사용되지 않기 때문에, 이러한 oob 샘플을 검증셋(validation set)이나 교차검증(cross validation)에 사용할 수 있다.

Scikit-Learn에서는 BaggingClassifier의 인자인 oob_score=True로 설정하면 학습이 끝난 후 자동으로 oob 평가를 할 수 있다.

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True, random_state=40)
bag_clf.fit(X_train_std, y_train)

print('oob score :', bag_clf.oob_score_)

In [ ]:
#아래의 코드와 같이 oob 샘플에 대한 결정 함수(decision function) 값(확률값)도 확인할 수 있다
bag_clf.oob_decision_function_[:5] 

랜덤 포레스트(Random Forest)는 배깅(bagging)을 적용한 의사결정나무(decision tree)의 앙상블이다. 
위에서 BaggingClassifier에 DecisionTreeClassifier를 인자로 넣어 줬었다. Scikit-Learn에서는 랜덤 포레스트를 간편하게 쓸 수 있도록 RandomForestClassifier를 제공 한다.

아래는 RandomForestClassifier 를 이용해 모델링하고, 배깅(bagging)을 적용한 의사결정나무(decision tree)의 앙상블과 비교한 것이다.

In [ ]:

# Random Forest
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rnd_clf.fit(X_train_std, y_train)
y_pred_rf = rnd_clf.predict(X_valid_std)

# BaggingClassifier
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16, random_state=42),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train_std, y_train)
y_pred = bag_clf.predict(X_valid_std)

# 두 모델의 예측 비교
print(np.sum(y_pred == y_pred_rf) / len(y_pred))

In [ ]:
print('Accuracy =', accuracy_score(y_valid, y_pred))

###### 엑스트라 트리 (Extremely Randomized Trees)
랜덤 포레스트는 트리를 생성할 때, 각 노드는 랜덤하게 특성(feature)의 서브셋(자식 노드를)을 만들어 분할한다. 반면 익스트림 랜덤 트리(Extremely Randomized Trees) 또는 엑스트라 트리(Extra-Trees)는 트리를 더욱 랜덤하게 생성하기 위해 노드를 분할하는 최적의 임계값을 찾는 것이 아니라 후보 특성을 이용해 랜덤하게 분할한 다음 그 중에서 최상의 분할을 선택하는 방법이다. 랜덤 포레스트 처럼 각 노드의 특성마다 최적의 임계값을 찾는것이 아니기 때문에 엑스트라 트리가 훨씬 학습 속도가 빠르다.

Scikit-Learn에서는 ExtraTreesClassifier을 이용해 엑스트라 트리를 구현할 수 있다.

아래는 랜덤포레스트와 엑스트라 트리를 비교한 것이다.

In [ ]:

from sklearn.ensemble import ExtraTreesClassifier

extra_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)
extra_clf.fit(X_train_std, y_train)
y_pred_ext = extra_clf.predict(X_valid_std)

# 두 모델의 예측 비교
print(np.sum(y_pred_rf == y_pred_ext) / len(y_pred_rf))

###### 모델 성능 튜닝

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())



In [ ]:
#RandomizedSearchCV 를 위한 파라미터 조건 설정

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 150, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

The most important arguments in RandomizedSearchCV are n_iter, which controls the number of different combinations to try, and cv which is the number of folds to use for cross validation (we use number of n_iter and number of cv respectively). More iterations will cover a wider search space and more cv folds reduces the chances of overfitting, but raising each will increase the run time. Machine learning is a field of trade-offs, and performance vs time is one of the most fundamental.

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 10,  
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train_std, y_train);

# n_iter, cv가 주요 하이퍼파라미터이다.

In [ ]:
rf_random.best_params_

In [ ]:
#사용할이름:  X_train_std, y_train  //  X_valid_std, y_valid  //  X_test_std, y_test

#### Evaluate the Default Model

In [ ]:
base_model = RandomForestClassifier(random_state = 42)
base_model.fit(X_train_std, y_train)
y_pred_tree = base_model.predict(X_valid_std)
print('Accuracy =', accuracy_score(y_valid, y_pred_tree))

#### Evaluate the Best Random Search Model

In [ ]:
best_random = rf_random.best_estimator_

y_pred_tree = best_random.predict(X_valid_std)
print('Accuracy =', accuracy_score(y_valid, y_pred_tree))


#### Grid Search

We can now perform grid search building on the result from the random search. We will test a range of hyperparameters around the best values returned by random search.

In [ ]:
rf_random.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [30, 50],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 4],
    'min_samples_split': [8,10],
    'n_estimators': [120,150]
}

# Create a base model
rf = RandomForestClassifier(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
# Fit the grid search to the data
grid_search.fit( X_train_std, y_train );

In [ ]:
grid_search.best_params_

#### Evaluate the Best Model from Grid Search

In [ ]:
best_grid = grid_search.best_estimator_

y_pred_tree = best_grid.predict(X_valid_std)
print('Accuracy =', accuracy_score(y_valid, y_pred_tree))

#### Final Model
The final model from hyperparameter tuning is as follows.

In [ ]:
final_model = grid_search.best_estimator_ # 위에서 valid set 평가시 성능 가장 좋았던 것을 고른다. (base_model,best_random,best_grid)

print('Final Model Parameters:\n')
pprint(final_model.get_params())
print('\n')

y_pred_tree = final_model.predict(X_test_std)
print('Accuracy =', accuracy_score(y_test, y_pred_tree)) #  test set에 적용한다.

In [ ]:
print("train set Accuracy:",final_model.score(X_train_std,y_train))
print("test set Accuracy:",final_model.score(X_test_std,y_test))

#### Evaluate Feature Significance
Investigating feature importance is a relatively straight forward process:

1.Out feature importance coefficients

2.Map coefficients to their feature name

3.Sort features in descending order

In [ ]:
# Fit model with training data
#clf.fit(X_train_std, y_train)

# Output feature importance coefficients, map them to their feature name, and sort values
coef = pd.Series(final_model.feature_importances_, index = X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 5))
coef.head(25).plot(kind='bar')
plt.title('Feature Significance')
plt.tight_layout()

In [ ]:
# using grid search model hyperparameter

# Output feature importance coefficients, map them to their feature name, and sort values
coef = pd.Series(best_grid.feature_importances_, index = X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 5))
coef.head(25).plot(kind='bar')
plt.title('Feature Significance')
plt.tight_layout()

In [ ]:
# Get numerical feature importances

feature_list = list(X_train.columns)
importances = list(best_grid.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');